## Embeddings in TensorFlow

在表示离散数值时如果用 one-hot 会很稀疏，效率不高。Embeddings 可通过训练来寻找单词之间的相似性。

In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import pandas as pd

tf.disable_eager_execution()

In [5]:
text = 'My cat is a great cat'
tokens = text.lower().split()
print(tokens)

['my', 'cat', 'is', 'a', 'great', 'cat']


In [7]:
vocab = set(tokens)
vocab = pd.Series(range(len(vocab)), index=vocab)
vocab

a        0
my       1
cat      2
is       3
great    4
dtype: int64

In [8]:
pd.get_dummies(tokens)

,a,cat,great,is,my
0,0,0,0,0,1
1,0,1,0,0,0
2,0,0,0,1,0
3,1,0,0,0,0
4,0,0,1,0,0
5,0,1,0,0,0


In [10]:
word_ids = vocab.loc[tokens].values
word_ids

array([1, 2, 3, 0, 4, 2])

### One-hot with TensorFlow

In [12]:
inputs = tf.placeholder(tf.int32, [None])

# TensorFlow has an operation for one-hot encoding
one_hot_inputs = tf.one_hot(inputs, len(vocab))

transformed = tf.Session().run(one_hot_inputs, {inputs: word_ids})
transformed

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]], dtype=float32)

### Embeddings with TensorFlow

通过 Embeddings representation (Embeddings 表示），每一个单词都可以被表示为一个向量，这个向量的长度就是 Embedding_size.

在这个例子中，我们将 embedding_size 的长度设置为 3, 即每个单词通过三个实数组成的向量来表示。

另外，在这个例子中 `embeddings` 是随机生成的，所以并不能表示单词之间的任何关系，只是借鉴 word2vec 的思想，如果在 NN 的训练过程中，embeddings 是会被更新的。


In [20]:
embedding_size = 3

inputs = tf.placeholder(tf.int32, [None], name='word_ids')

# This is where the embedding vecots live 
# This will be modified by the optimiztion unless trainable=False
# I choose random normal distribution but you can try other distribution
embeddings = tf.random_normal(shape=(len(vocab), embedding_size))

embedded = tf.nn.embedding_lookup(embeddings, inputs)

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
transformed = sess.run(embedded, {inputs: word_ids})
print(transformed)

[[-0.13878635  0.86863303 -1.6017141 ]
 [-0.21395133 -1.1467001   0.24122861]
 [ 1.9225732  -1.6799307   0.5026513 ]
 [-0.10037646  0.54069585  1.7154973 ]
 [-1.526876   -1.6427577  -0.0230563 ]
 [-0.21395133 -1.1467001   0.24122861]]


In [21]:
sess.run(embeddings)

array([[ 1.5956285 ,  1.3354356 , -0.23255636],
       [ 0.859987  , -0.64641064, -0.65326685],
       [ 1.0356464 ,  1.4639943 , -0.17944591],
       [-0.86762327, -0.05347135, -0.8178174 ],
       [-0.7942827 ,  0.3838643 ,  0.88105506]], dtype=float32)

In [22]:
sess.run(embedded, {inputs: [0, 2]})

array([[-0.42670134, -0.26926398,  0.26791432],
       [ 0.05090201, -1.3788782 ,  0.18103495]], dtype=float32)

### Pretrained embeddings

像上面我们随机初始化的 embeddings，向量之间是没有任何关系的。为了寻找向量之间的相似性，我们可可以使用 word2vec 训练它们，或者使用已经训练好的向量。


#### References
1. [Embeddings with TensorFlow](https://post2web.github.io/posts/embeddings-with-tensorflow/)